# COVID-19: the Politics and Efficacy of Lockdown

I am curious about COVID-19, and this notebook is my effort to find context and perspective from responsible, public data.

All data used is from:

- [The *New York Times*](https://github.com/nytimes/covid-19-data)
- [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm)
- [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm)
- [South Carolina Department of Health and Environmental Control (DHEC)](https://www.scdhec.gov/vital-records/parentage/sc-vital-records-data-and-statistics)
- [The Office of National Statistics, UK](https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/weeklyprovisionalfiguresondeathsregisteredinenglandandwales)
- [Civil Services State Governors Data](https://github.com/CivilServiceUSA/us-governors)
- [A map of the US cities and states under lockdown — and those that are reopening. Business Insider (updated as of May 5)](https://www.businessinsider.com/us-map-stay-at-home-orders-lockdowns-2020-3)
- [See Which States Are Reopening and Which Are Still Shut Down](https://www.nytimes.com/interactive/2020/us/states-reopen-map-coronavirus.html)
- [2016 United States Presidential Election. Wikipedia](https://en.wikipedia.org/wiki/2016_United_States_presidential_election)

The top part of this notebook consists of configurations. The more interesting stuff is below. I am not a professional any of the domains that this notebook touches on, so I make no claims.

Furthermore, I know that the data, even that from respectable sources, is all over the place, so any model, and especially amateur models like the present one, are pretty much guaranteed to be wrong. See [this very clear presentation](https://xkcd.com/2295/).

If there is any value here, it is in taking the same data that we see all over the place, and telling different stories with it.

## Configuring Libraries for the Almond Kernel

First, we'll make a bintray repository with libraries available to the almond kernel.

In [ ]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")

interp.repositories() ++= Seq(myBT)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
init(offline=true)

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 8)

## Imports

From this point on, the notebook consists of completely generic Scala.

In [ ]:
import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source

import java.text.SimpleDateFormat
import java.util.Date


## Useful Functions

Pretty Print Things:

In [ ]:
def showMe(v:Any):Unit = {
  v match {
    case _:Vector[Any] => println(s"""\n----\n${v.asInstanceOf[Vector[Any]].mkString("\n")}\n----\n""")
    case _:Iterable[Any] => println(s"""\n----\n${v.asInstanceOf[Iterable[Any]].mkString("\n")}\n----\n""")
    case _ => println(s"\n-----\n${v}\n----\n")
  }
}

Validate an `Option[String]` as a valid date in 2020, or `None`.

In [ ]:
def isValidDate( dOpt: Option[String]): Boolean = {
    
    val monthMap: Map[Int, Int] = {
        Map(1 -> 31,
        2 -> 29,
        3 -> 31,
        4 -> 30,
        5 -> 31,
        6 -> 30,
        7 -> 31,
        8 -> 31,
        9 -> 30,
        10 -> 31,
        11 -> 30,
        12 -> 31)
    }
    
    dOpt match {
        case Some(dateString) => {
            val y: Int = dateString.split("-")(0).toInt
            val m: Int = dateString.split("-")(1).toInt
            val d: Int = dateString.split("-")(2).toInt
            
            val goodYear = (y == 2020)
            val goodMonth = ( (1 to 12).contains(m) )
            val goodDay = {
                (d >= 1) &
                (d <= monthMap(m))
            }
            (goodYear & goodMonth & goodDay)
        }
        case None => true
    }
    
}

## Load Some Data

Load up-to-date data from the NY Times. Source: <https://github.com/nytimes/covid-19-data>.

In [ ]:
val dataLines: Vector[String] = {
    scala.io.Source.fromURL("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv").mkString.split("\n").toVector.tail
}

// quick test
val badLines = dataLines.filter( l => {
    l.split(",").size != 5
})

assert ( badLines.size == 0 )


Let's make an indexed Vector of dates… it will be easier to work with later…

In [ ]:
val listOfDates1: Vector[String] = dataLines.map(_.split(",").head).toVector.distinct

val listOfDates2: Vector[String] = Vector(
    "2020-05-19",
    "2020-05-20",
    "2020-05-21",
    "2020-05-22",
    "2020-05-23",
    "2020-05-24",
    "2020-05-25",
    "2020-05-26",
    "2020-05-27",
    "2020-05-28",
    "2020-05-29",
    "2020-05-30",
    "2020-05-31"
)

val listOfDates = (listOfDates1 ++ listOfDates2).distinct

// showMe(listOfDates)

A separate data-file, `politics_and_lockdown.csv`, to be merged with our day-by-day report from the NYT.

We'll make a class, `StateStat`, to organize this data. We'll save a `StateStat` for each state in `stateStats: Vector[StateStat]`.

That file's field-headings are:

~~~
state,population,politics,total-lockdown,partial-lockdown,opening
~~~

In [ ]:
case class StateStat( 
    state:String, 
    population: Int, 
    isRedState: Boolean,
    partialLock: Option[String], 
    totalLock: Option[String],
    open: Option[String]
)

val stateStats: Map[String, StateStat] = {
    val fileName = "politics_and_lockdown.csv"
    scala.io.Source.fromFile(fileName).mkString.split("\n").toVector.tail.map( l => {
        val state: String = l.split(",").toVector(0)
        val population: Int = l.split(",").toVector(1).toInt
        val isRedState: Boolean = l.split(",").toVector(2).trim match {
            case "republican" => true
            case _ => false
        }
        val partialLock = l.split(",").toVector(4) match {
            case "none" => None
            case _ => Some(l.split(",").toVector(4))
        }
        val totalLock = l.split(",").toVector(3) match {
            case "none" => None
            case _ => Some(l.split(",").toVector(3))
        }
        val open = l.split(",").toVector(5) match {
            case "none" => None
            case _ => Some(l.split(",").toVector(5))
        }
        (state, StateStat(state, population, isRedState, partialLock, totalLock, open))
    }).toMap
}

// Let's do a little sanity-checking on the dates…


val invalidDates: Vector[StateStat] = {
    stateStats.toVector.filter( ss => {
        val p: Option[String] = ss._2.partialLock
        val t: Option[String] = ss._2.totalLock
        val o: Option[String] = ss._2.open
        
        val bp: Boolean = isValidDate(p)
        val bt: Boolean = isValidDate(t)
        val bo: Boolean = isValidDate(o)
        
        val pInList: Boolean = {
            p match {
                case Some(s) => listOfDates.contains(s)
                case None => true
            }
        }
        val tInList: Boolean = {
            t match {
                case Some(s) => listOfDates.contains(s)
                case None => true
            }
        }
        val oInList: Boolean = {
            o match {
                case Some(s) => listOfDates.contains(s)
                case None => true
            }
        }
        
        val isInList: Boolean = (pInList & tInList & oInList)
        
        ((bp & bt & bo & isInList) == false)
    }).map(_._2)
}

if (invalidDates.size > 0) showMe(invalidDates)

assert( invalidDates.size == 0)

## Make Data Structures

For each day reported for each state, we want the following information:

- Date
- State
- State population (based on 2018 data)
- Red state? (a `boolean`). Based on party of the governor, with two exceptions described below
- New Deaths
- New Cases
- Total Deaths by this day
- Total Cases by this day
- Lockdown state on this date (based on [this article](https://www.nytimes.com/interactive/2020/us/states-reopen-map-coronavirus.html))
    - `0` = no lockdown, or lockdown lifted
    - `1` = partial state lockdown
    - `2` = state lockdown
    
States reported their first cases on different dates, of course, so we'll need to pad the data on the left so they all start on the same day (January 21, 2020, when Washington reported its first case).

Since “open”, “closed”, “locked-down”, *etc.* are vague terms, I am calling any state with a “Stay at Home” order to be fully closed, and any state that allows indoor restaurant dining to be fully open, as these things go.

### A Note on Red vs. Blue

The file `politics_and_lockdown.csv` in this repository lists states, with their 2018 populations, and their political leaning. The latter category is of course slippery. I based this version on (1) how the state voted in the 2016 presidential election, but also (2) the political party of the governor. I used my own, fallible, subjective judgement about which category to use, when (1) and (2) disagreed. So, Massachusetts was very, very blue in 2016, and I left it that way despite the state's having a Republican governor. On the other hand, Michigan was a red-state in 2016, but its Democratic governor seems to be playing a very dominant role in that state's response to the crisis, so Michigan is "blue" in this chart.

If you disagree, please edit the file `politics_and_lockdown.csv`, and re-run the scripts to see what you see!




In [ ]:
case class StateDay(
    date: String,
    state: String,
    population: Int,
    isRedState: Boolean,
    newDeaths: Int,
    newCases: Int,
    totalDeaths: Int,
    totalCases: Int,
    activeCases: Int,
    lockdown: Int
)

First, let's split out our raw data so each state's data is in a Vector[String]. We'll make a Vector of those: `Vector[Vector[String]]`. This will let us do the initial normalization (padding left so they all start on `2020-01-21').

We can do some consistency-checking while we're at it.

In [ ]:
// For each state, a vector of that state's daily records, from the NYTimes
val dataVec1: Vector[Vector[String]] = { 
    // get just the states
    val justStates: Vector[String] = dataLines.map( dl => {
        dl.split(",")(1)
    })
    // map dataLines to states
    val zippedWithState: Vector[(String, String)] = justStates.zip(dataLines)
    // group by state
    val interim1: Vector[(String, Vector[(String, String)])] = {
        zippedWithState.groupBy(_._1).toVector
    }
    // simplify
    val interim2: Vector[Vector[String]] = interim1.map( i1 => {
        (i1._1, i1._2.map(_._2))
    }).map(_._2)
    // sort, first by date within state, then by number of days
    interim2.map( i2 => {
        i2.sortBy(_.split(",")(0))
    }).sortBy(_.size)
}

// Let's filter out everything but the 50 states…
val justThe50: Vector[Vector[String]] = dataVec1.filter( dv => {
    val fiftyStates: Vector[String] = stateStats.keys.toVector
    fiftyStates.contains(dv.head.split(",")(1))
})



## Build the Real Data

For reference:

~~~
case class StateStat( 
    state:String, 
    population: Int, 
    isRedState: Boolean,
    partialLock: Option[String], 
    totalLock: Option[String],
    open: Option[String]
)
~~~

And what we want is a Vector[Vector[StateDay]]:

~~~
case class StateDay(
    date: String,
    state: String,
    population: Int,
    isRedState: Boolean,
    newDeaths: Int,
    newCases: Int,
    totalDeaths: Int,
    totalCases: Int,
    activeCases: Int,
    lockdown: Int
)
~~~

We start by mapping `justThe50` (above) to a Vector[Vector[StateDay]], which will require some helper-fnctions, and merging it with `stateStats`.

Because calculating the lockdown data for a given day is a little fiddly, let's make a first-cut, getting everything into a good data-structure, easier to work with than a delimited String, then go back and sort out lockdown data. 

In [ ]:
def oneStateVector( stats: Map[String, StateStat] = stateStats, sv: Vector[String]): Vector[StateDay] = {
    
    /*
    // Make sure all dates are valid
    val invalidDates: Vector[String] = {
        sv.map(_.split(",")(0)).filter( d => (isValidDate(d) == false)
    }
                                       
    if (invalidDates > 0) {
        println( sv.head )
        showMe(invalidDates)
    }
                                       
    assert( invalidDates == 0) // stop if there is bad data
    */
    
    // Make sure we've sorted by date
                                       
    val sortedSv = sv.sortBy( d => {
        d.split(",")(0)
    })                           
                                
                                       
                                       
    sortedSv.zipWithIndex
            .map( recString => {
                val fields: Vector[String] = recString._1.split(",").toVector
                val date: String = fields(0)
                val state: String = fields(1)
                val totalCases: Int = fields(3).toInt // NYT gives a running toll
                val totalDeaths: Int = fields(4).toInt // NYT gives a running toll
                
                val i = recString._2
                
                // By subtraction from the previous totalCases, get newCases
                val newCases: Int = {
                    if (i == 0) totalCases
                    else {
                        val totalToday: Int = totalCases
                        val totalPrev: Int = {
                            sortedSv(i-1).split(",")(3).toInt
                        }
                        totalToday - totalPrev
                    }
                }
                
                // By subtraction from the previous totalDeaths, get newDeaths
                val newDeaths: Int = {
                    if (i == 0) totalDeaths
                    else {
                        val totalToday: Int = totalDeaths
                        val totalPrev: Int = {
                            sortedSv(i-1).split(",")(4).toInt
                        }
                        totalToday - totalPrev
                    }
                }
                
                // Get the easy stuff from stats…
                val population: Int = stats(state).population
                val isRedState: Boolean = stats(state).isRedState
                
                // Get today's lockdown state!
                val lockdown: Int = 0
                
                // We'll fill in activeCases later!
                val activeCases: Int = 0
                
                StateDay( date,
                         state,
                         population,
                         isRedState,
                         newDeaths,
                         newCases,
                         totalDeaths,
                         totalCases,
                         activeCases,
                         lockdown
                        )
            })
}



val dailyStateRecords_invalidLockdownData: Vector[Vector[StateDay]] = {
    justThe50.map( s => {
        oneStateVector( stateStats, s)
    })
}



## Padding the Days

Different states started reporting on different dates. We need all of our states Vectors of `StateDay` objects to be the same length, so we need to pad them with "empty" records on the "left". We find the state that has the longest reporting record (Washington, but we'll find it programmatically), and use its records as the default length. We'll padd all the others to match.

As we do this, remember:

~~~
case class StateDay(
    date: String,
    state: String,
    population: Int,
    isRedState: Boolean,
    newDeaths: Int,
    newCases: Int,
    totalDeaths: Int,
    totalCases: Int,
    activeCases: Int,
    lockdown: Int
)
~~~

We will use `listOfDates`, a `Vector[String]` defined above, and a representative `StateDay` to give us the data we need.

In [ ]:
def blankDayVec( howMany: Int, state: StateDay ): Vector[StateDay] = {
    val dateVec: Vector[String] = listOfDates.toVector.take(howMany)
    dateVec.map( date => {
        StateDay(
            date,
            state.state,
            state.population,
            state.isRedState,
            0,0,0,0,0,0
        )
    })
}

// Sort the states by number of days, take the size of the biggest
val maxRecords: Int = dailyStateRecords_invalidLockdownData.sortBy(_.size).last.size

val dailyStateRecords_normalized: Vector[Vector[StateDay]] = {
    dailyStateRecords_invalidLockdownData.map( stateVec => {
        val size: Int = stateVec.size
        val diff: Int = maxRecords - size
        if (diff == 0) stateVec
        else {
            val paddingVec: Vector[StateDay] = {
                blankDayVec( diff, stateVec.head )
            }
            paddingVec ++ stateVec
        }
    })
}

// Check that it worked! 

assert ( dailyStateRecords_normalized.map(_.size).distinct.size == 1 )

// Un-comment the line below to see the result
//for ( s <- dailyStateRecords_normalized)  showMe(s)



Now we can take `dailyStateRecords_invalidLockdownData` and map it to another `Vector[Vector[StateDay]]` with valid data for the lockdown state on each day. We will use `listOfDates`, a `Vector[String]` defined above.

We'll start with a Class `LockdownEvent`, which will have to parameters: a day-index, and a lockdown state (0,1,2) as described above: 0 = no lockdown or full open, 1 = partial lockdown, 2 = full lockdown.

In [ ]:
case class LockdownEvent( dayIndex: Int, lockdownState: Int )

def getLockdownState( 
    record: StateDay, 
    stateRecords: Vector[StateDay], 
    stats: Map[String, StateStat] = stateStats): Int = {
    
    val i: Int = stateRecords.indexOf(record)
    
    // every state started at lockdown = 0
    if (i == 0) { 0 }
    /* 
        otherwise, we get the date of partial-lockdown, total-lockdown, and opening
        and see which was the most recent previous event.
    */
    else {
        val sstat: StateStat = stats(record.state)
        val partLock: Option[String] = sstat.partialLock
        val totalLock: Option[String] = sstat.totalLock
        val open: Option[String] = sstat.open
        
        
        /*
        Now we get the dateIndex for this record
        */
        
        val todayIndex: Int = listOfDates.indexOf(record.date)
        //println(s"${record.state} :: ${i} :: ${todayIndex}")
        
        /*
        For each event (lockdown states 0,1,2, see above), let's make a LockdownEvent object.
        If an event is None, we put it at Day 0.
        */
        
        
        val partLockEvent: LockdownEvent = {
            partLock match {
                case Some(d) => {
                    val dayIndex: Int = listOfDates.indexOf(d)
                    LockdownEvent(dayIndex, 1)
                }
                case None => LockdownEvent(0, 0)
            }
        }
        val totalLockEvent: LockdownEvent = {
            totalLock match {
                case Some(d) => {
                    val dayIndex: Int = listOfDates.indexOf(d)
                    LockdownEvent(dayIndex, 2)
                }
                case None => LockdownEvent(0, 0)
            }
        }
        val openEvent: LockdownEvent = {
            open match {
                case Some(d) => {
                    val dayIndex: Int = listOfDates.indexOf(d)
                    LockdownEvent(dayIndex, 0)
                }
                case None => LockdownEvent(0, 0)
            }
        }
      //println(s"${totalLockEvent} :: ${partLockEvent} :: ${openEvent}")

        
        // We make a Vector of these, filter for earler ones, and take the last.
        val eventVec: Vector[LockdownEvent] = {
            Vector( partLockEvent, totalLockEvent, openEvent ).sortBy( e => {
                e.dayIndex
            })
        }
        
        //for ( e <- eventVec) println( e )
        
       // showMe(todayIndex)
       // showMe(eventVec)
        
        val mostRecent: LockdownEvent = {
            val validEvents: Vector[LockdownEvent] = eventVec.filter( _.dayIndex <= todayIndex)
            if (validEvents.size > 0) validEvents.last
            else LockdownEvent(todayIndex, 0)
        }
        
        //println(s"${record.state} :: ${todayIndex} :: ${mostRecent.lockdownState}")
            
        // return the lockdown State
        mostRecent.lockdownState
        
    }
}



In [ ]:
// And we'll sort by State-name while we're at it
val dailyStateRecords_unsorted: Vector[Vector[StateDay]] = {
    dailyStateRecords_normalized.map( vsd => {
        vsd.map( record => {
            val lockdownState: Int = getLockdownState( record, vsd )
            StateDay( record.date,
                     record.state,
                     record.population,
                     record.isRedState,
                     record.newDeaths,
                     record.newCases,
                     record.totalDeaths,
                     record.totalCases,
                     record.activeCases,
                     lockdownState
                    )
        })
    })
}


In [ ]:

val dailyStateRecords_sorted: Vector[Vector[StateDay]] = dailyStateRecords_unsorted.sortBy(_.head.state)

// Un-comment the line below to see the data…
//for ( s <- dailyStateRecords_sorted)  showMe(s)


//for ( s <- dailyStateRecords_sorted.filter(_.head.state == "Utah"))  showMe(s)

## Calculate Active Cases

If you have been paying attention, you will have seen that we punted on "active cases". No one reports these consistently, so we'll have to calculate them for each State's daily data. In other words, we'll make something up.

But we couldn't make something up until we had a normalized dataset, with the same number of days for each state, so we've saved this for last.

We can assume that there is some **n**, where **n = days-until-death-or-recovery**. I am not a physician or epidemiologist, so I'll just have to make up a number.

That number is defined below, as `activeCaseThreshold`. It is an `Int` representing a number of days. 

The way it will work is this: Any reported case that is farther in the past than **n-days** will be considered *inactive*. The difference between the current *total cases* and these calculated *inactive cases* will be the **active cases**.

You can change it, and change the results!

In [ ]:
val activeCaseThreshold: Int = 14

So let's make one final version of our `Vector[Vector[StateDay]]` with calculated active cases.

In [ ]:
val stateRecords: Vector[Vector[StateDay]] = {

    dailyStateRecords_sorted.map ( vsdd => {

        vsdd.zipWithIndex.map( isd => {
            val i: Int = isd._2
            val record: StateDay = isd._1

            val activeCases: Int = {
                if (i <= activeCaseThreshold) record.totalCases
                else {
                    val inactive = vsdd(i - activeCaseThreshold).totalCases
                    record.totalCases - inactive
                }
            }

            StateDay( record.date,
                     record.state,
                     record.population,
                     record.isRedState,
                     record.newDeaths,
                     record.newCases,
                     record.totalDeaths,
                     record.totalCases,
                     activeCases,
                     record.lockdown
                    )


        })

    })
}

// Un-comment the line below to see the result
//for ( s <- dailyStateRecords)  showMe(s)


## A More Clear Data Structure

A `Vector[Vector[StateDay]]` is going to get confusing, so let's make a Class with better names.

What we has is an outer Vector containg data for states. For each state, we have StateDays.

In [ ]:
case class OneStateData( days: Vector[StateDay]) 

case class StatesData( states: Vector[OneStateData] )

val statesData: StatesData = {
    val allStates: Vector[OneStateData] = stateRecords.map( ss => {
        OneStateData( ss )
    })
    StatesData( allStates )
}

def getAState( state: String, data: Vector[OneStateData] = statesData.states): OneStateData = {
    val sr: Vector[OneStateData] = data.filter(_.days.head.state == state)
    sr.head
}

//showMe(getAState("Utah").days)

## Ready to Work

At this point, we have a "list of lists" in the form of `dailyStateRecords`. The top-level list is the 50 states plus the District of Columbia. We will test this.

For each state, we have a list of daily reports, each report showing:

~~~
case class StateDay(
    date: String,
    state: String,
    population: Int,
    isRedState: Boolean,
    newDeaths: Int,
    newCases: Int,
    totalDeaths: Int,
    totalCases: Int,
    activeCases: Int,
    lockdown: Int
)
~~~



In [ ]:
// last tests

// There should be 51
assert( statesData.states.size == 51 )

// They should all be the same size
assert( statesData.states.map(_.days.size).distinct.size == 1)


# Questions


Now that we've spent hundreds of lines of code sorting things out, we can ask questions:

- Which states have been the most, and which the least, "locked down"? (This should require undoing the normalization we did above, since it isn't fair to give states credit or blame for being "not locked down" before they had any cases.)
- How does that map to politics?
- How does degree of lockdown map to cases and deaths?
- How does degree of lockdown map to Case Fatality Rate?


(Please read, above, where I describe my subjective mapping of states to "red" or "blue". You can edit my state mapping to get other views.)


### Degree of Lockdown

We want to take each state's data, truncate it on the left with the first reported case, and average its lockdown-degree, as an average of "lockdown state per day". We'll multiply by 100 so we can work with 3 significant figures while still doing integer arithmetic. States that never locked down at all will have an average of 0 (see Arkansas, for example). If a hypothetical state fully locked down upon reporting of the first case, it would have an average of 200 (Delaware comes closest, as of 5/12/2020).

In [ ]:
val sr: Vector[OneStateData] = statesData.states.filter(_.days.head.state == "California")

sr.size

In [ ]:


def oneStateLockdownScore( state: String, report: Boolean = true): Int = {
    val stateRecords: OneStateData = statesData.states.filter(_.days.head.state == state).head
    if (report) println(s"\n-----\n${state}")
    val stateFirstCaseRecord = stateRecords.days.find( _.newCases > 0 ).get
    val stateFirstCaseRecordIndex = stateRecords.days.indexOf(stateFirstCaseRecord)
    val stateRecordsClipped = stateRecords.days.takeRight(stateRecords.days.size - stateFirstCaseRecordIndex).filter(_.state == state) 
    //showMe(stateRecordsClipped)
    if (report) println(s"Days since first case: ${stateRecordsClipped.size}")
    val stateLDScores = stateRecordsClipped.map( _.lockdown * 100 )
    //println(s"State LD Scores: ${state}")
    //showMe(stateLDScores)
    val stateClippedSize = stateRecordsClipped.size
    val stateLDAverage = stateLDScores.sum / stateClippedSize
    if (report) println(s"${stateLDScores.sum} /  ${stateRecordsClipped.size} = ${stateLDAverage}\n-----")
    
    stateLDAverage
}

//oneStateLockdownScore( "Utah", true)
//oneStateLockdownScore( "New York", true)

val stateScores: Vector[(String, Int)] = statesData.states.map( dsr => {
    val state: String = dsr.days.head.state
    (state, oneStateLockdownScore(state, false))
}).sortBy(_._2)

//Un-comment the code block below to list states by degree of lockdown


for (ss <- stateScores.zipWithIndex) {
    val rank = ss._2 + 1
    val state = ss._1._1
    val score = ss._1._2
    println(s"${rank}. ${state} ${score}")
}





## Generally Useful Functions

Our basic data structure is a **list of lists of `StateDay` objects**. We might want to slice and dice this in various ways: "red" states vs. "blue" states, states at various degrees of lockdown, comparing particular states, etc.

All of this we can do with filtering our **list of lists**.

Let's make some functions that will take a `Vector[Vector[StateDay]]` and do some generically useful things to it.

In [ ]:
val perCapitaNumber: Int = 100000 // Change this!

def mountingDeaths( sd: StatesData, perCapita: Boolean = false ): Vector[Double] = {
    if (perCapita) {
         val population: Int = sd.states.map( _.days.head.population ).sum
         (0 until sd.states.head.days.size).toVector.map( dayIndex => {
            val sumDeaths: Int = sd.states.map( state => {
                state.days(dayIndex).totalDeaths
            }).sum 
            (sumDeaths.toDouble * perCapitaNumber.toDouble) / population.toDouble
        })
    } else {
        (0 until sd.states.head.days.size).toVector.map( dayIndex => {
            sd.states.map( state => {
                state.days(dayIndex).totalDeaths
            }).sum.toDouble
        })
    }
}

def dailyDeaths( sd: StatesData, perCapita: Boolean = false ): Vector[Double] = {
    if (perCapita) {
         val population: Int = sd.states.map( _.days.head.population ).sum
         (0 until sd.states.head.days.size).toVector.map( dayIndex => {
            val sumDeaths: Int = sd.states.map( state => {
                state.days(dayIndex).newDeaths
            }).sum 
            (sumDeaths.toDouble * perCapitaNumber.toDouble) / population.toDouble
        })
    } else {
        (0 until sd.states.head.days.size).toVector.map( dayIndex => {
            sd.states.map( state => {
                state.days(dayIndex).newDeaths
            }).sum.toDouble
        })
    }
}

def mountingCases( sd: StatesData, perCapita: Boolean = false ): Vector[Double] = {
    if (perCapita) {
         val population: Int = sd.states.map( _.days.head.population ).sum
         (0 until sd.states.head.days.size).toVector.map( dayIndex => {
            val sumCases: Int = sd.states.map( state => {
                state.days(dayIndex).totalCases
            }).sum 
            (sumCases.toDouble * perCapitaNumber.toDouble) / population.toDouble
        })
    } else {
        (0 until sd.states.head.days.size).toVector.map( dayIndex => {
            sd.states.map( state => {
                state.days(dayIndex).totalCases
            }).sum.toDouble
        })
    }
}

def dailyCases( sd: StatesData, perCapita: Boolean = false ): Vector[Double] = {
    if (perCapita) {
         val population: Int = sd.states.map( _.days.head.population ).sum
         (0 until sd.states.head.days.size).toVector.map( dayIndex => {
            val sumCases: Int = sd.states.map( state => {
                state.days(dayIndex).newCases
            }).sum 
            (sumCases.toDouble * perCapitaNumber.toDouble) / population.toDouble
        })
    } else {
        (0 until sd.states.head.days.size).toVector.map( dayIndex => {
            sd.states.map( state => {
                state.days(dayIndex).newCases
            }).sum.toDouble
        })
    }
}



## Categories

We can do Red States vs. Blue States, groups by degree of lockdown, or individual states.

In [ ]:
val redStates: StatesData = {
    val states: Vector[OneStateData] = statesData.states.filter(_.days.head.isRedState == true)
    StatesData(states)
}

val blueStates: StatesData = {
    val states: Vector[OneStateData] = statesData.states.filter(_.days.head.isRedState == false)
    StatesData(states)
}

val lockdownThirds: Map[String, StatesData] = {
    val howMany: Int = stateScores.size / 3
    val statesRanked: Vector[String] = stateScores.map(_._1)
    val lowStates: StatesData = {
        val chosenStates: Vector[OneStateData] = {
            val stateList: Vector[String] = statesRanked.take(howMany)
            statesData.states.filter( s => {
                stateList.contains(s.days.head.state)
            })
        }
        StatesData(chosenStates)
    }
    
    val middleStates: StatesData = {
        val chosenStates: Vector[OneStateData] = {
            val stateList: Vector[String] = statesRanked.takeRight(howMany * 2).take(howMany)
            statesData.states.filter( s => {
                stateList.contains(s.days.head.state)
            })
        }
        StatesData(chosenStates)
    }
    
    val highStates: StatesData = {
        val chosenStates: Vector[OneStateData] = {
            val stateList: Vector[String] = statesRanked.takeRight(howMany)
            statesData.states.filter( s => {
                stateList.contains(s.days.head.state)
            })
        }
        StatesData(chosenStates)
    }
    
    Map("low" -> lowStates, "middle" -> middleStates, "high" -> highStates)
}



def pickAState( state: String ): StatesData = {
    val states: Vector[OneStateData] = statesData.states.filter(_.days.head.state == state)
    StatesData(states)
}



### Graph Things

Let's write a parameterized function for graphing a `Vector[Vector[Int]]`, so we can throw at it any combination of lists and see what we can see. We'll want to parameterized "title" as well as a `Vector[String]` that are labels for each of our data vectors.


In [ ]:

def getScatterVec( data: Vector[Vector[Double]], lables: Vector[String], color: Option[(Int, Int, Int, Double)] = None): Vector[Scatter] = {
    data.zipWithIndex.map( vi => {
        
        val colorList: Vector[(Int, Int, Int, Double)] = {
            Vector((240,0,0,1.0),(0,0,240,1.0),(0,240,0,1.0),(2,63,165,1.0),(125,135,185,1.0),(190,193,212,1.0),(214,188,192,1.0),(187,119,132,1.0),(142,6,59,1.0),(74,111,227,1.0),(133,149,225,1.0),(181,187,227,1.0),(230,175,185,1.0),(224,123,145,1.0),(211,63,106,1.0),(17,198,56,1.0),(141,213,147,1.0),(198,222,199,1.0),(234,211,198,1.0),(240,185,141,1.0),(239,151,8,1.0),(15,207,192,1.0),(156,222,214,1.0),(213,234,231,1.0),(243,225,235,1.0),(246,196,225,1.0),(247,156,212,1.0))
        }
        
        val index = vi._2
        val dataVec = vi._1
        val label = lables(index)
        val thisColor = {
            color match {
                case Some(tp) => tp
                case None => colorList( index % colorList.size )
            }
        }
    
        Scatter(
          (1 to dataVec.size),
          dataVec,
          name = label,
          mode = ScatterMode(ScatterMode.Lines),
          marker = Marker(
            color = Color.RGBA(
            thisColor._1,
            thisColor._2,
            thisColor._3,
            thisColor._4
           ),
          )
        )    
    })
}

def plotData( data: Vector[Vector[Double]], labels: Vector[String], title: String ): Unit = {
    
    val dataNew = getScatterVec( data, labels)
    val layoutNew = Layout(title)

    plot(dataNew, layoutNew)
}



# Visualizations!

Finally, after all this setup, we can do some actual visualizations!

In [ ]:
val redBluePerCap: Unit = {
    val data: Vector[Vector[Double]] = Vector(
        mountingDeaths(redStates, true), mountingDeaths(blueStates, true)
    )
    val labels = Vector("Red States", "Blue States")
    val title = s"Mounting Deaths, Red vs. Blue, per ${perCapitaNumber}"
    
    plotData(data, labels, title )
}

val redBlueDailyPerCap: Unit = {
    
    val rollingAve: Int = 14 // change to 1 for raw data

    val data: Vector[Vector[Double]] = Vector(
        dailyDeaths(redStates, true).sliding(rollingAve,1).toVector.map(_.sum / rollingAve), 
        dailyDeaths(blueStates, true).sliding(rollingAve,1).toVector.map(_.sum / rollingAve)
    )
    val labels = Vector("Red States", "Blue States")
    val title = s"Daily Deaths, Red vs. Blue, per ${perCapitaNumber}, ${rollingAve} day ave."
    
    plotData(data, labels, title )
}



val lockdownDeathTollPerCap: Unit = {
    
   val data = lockdownThirds.toVector.reverse.map( ldt => {
        mountingDeaths(ldt._2, true)
    })
    
   val lables = lockdownThirds.toVector.reverse.map( _._1 )

    
    val labels = Vector("Least Locked down 17 states", "Middle 17 states", "Most Locked down 17 states").reverse
    val title = s"Mounting Deaths, by degree of lockdown, per ${perCapitaNumber}"
    
    plotData(data, labels, title )
}

// This uses a rolling average, parameterized below…

val lockdownDailyDeathsPerCap: Unit = {
    
   val rollingAve: Int = 7 // change to 1 for raw data
    
   val data = lockdownThirds.toVector.reverse.map( ldt => {
        dailyDeaths(ldt._2, true).sliding(rollingAve,1).toVector.map(_.sum / rollingAve)
    })
    
   val lables = lockdownThirds.toVector.map( _._1 )

    
    val labels = Vector("Least Locked down 17 states", "Middle 17 states", "Most Locked down 17 states").reverse

    val title = s"Daily Deaths, by degree of lockdown, per ${perCapitaNumber}, ${rollingAve}-day avg."
    
    plotData(data, labels, title )
}

// This uses a rolling average, parameterized below…

val lockdownDailyCasesPerCap: Unit = {
    
   val rollingAve: Int = 7 // change to 1 for raw data
    
   val data = lockdownThirds.toVector.reverse.map( ldt => {
        dailyCases(ldt._2, true).sliding(rollingAve,1).toVector.map(_.sum / rollingAve)
    })
    
   val lables = lockdownThirds.toVector.map( _._1 )

    
    val labels = Vector("Least Locked down 17 states", "Middle 17 states", "Most Locked down 17 states").reverse

    val title = s"Daily Cases, by degree of lockdown, per ${perCapitaNumber}, ${rollingAve}-day avg."
    
    plotData(data, labels, title )
}

## A Complicated Thing

Let's see, day by day, how many people were under the three states of "lockdown".

We want three objects, "open", "partial", and "full" lockdown, each consisting of a `Vector`, which will be each day. It will consist of a `Vector[StateDay]`, depending on the state of lockdown for each state on each day.

This will let us see the population on different states of lockdown over time, and how those conditions may have mapped to cases and deaths.

This will assume that on each day, there is at least one state in each state of lockdown. We may need to come back and do some more padding at the pandemic winds down.

In [ ]:
val dailyLockdownPicture: Map[String, Vector[Vector[StateDay]]] = {
    val howMany: Int = statesData.states.head.days.size
    val zeroDays: Vector[Vector[StateDay]] = (0 until howMany).toVector.map( day => {
        val eachDayData: Vector[StateDay] = {
            statesData.states.map( sd => {
                sd.days(day)
            }).filter(_.lockdown == 0)
        }
        eachDayData
    })
     val oneDays: Vector[Vector[StateDay]] = (0 until howMany).toVector.map( day => {
        val eachDayData: Vector[StateDay] = {
            statesData.states.map( sd => {
                sd.days(day)
            }).filter(_.lockdown == 1)
        }
        eachDayData
    })
     val twoDays: Vector[Vector[StateDay]] = (0 until howMany).toVector.map( day => {
        val eachDayData: Vector[StateDay] = {
            statesData.states.map( sd => {
                sd.days(day)
            }).filter(_.lockdown == 2)
        }
        eachDayData
    })
    Map( "zero" -> zeroDays, "one" -> oneDays, "two" -> twoDays )
}



## Graphing

Now we can graph the total population under degrees of lockdown, and the per-capita new cases and deaths each day, sorted by degree of lockdown.

   

In [ ]:
val populationCurve: Vector[Vector[Int]] = {
    val zeroPop: Vector[Int] = dailyLockdownPicture("zero").map(day => day.map(_.population).sum)
    val onePop: Vector[Int] = dailyLockdownPicture("one").map(day => day.map(_.population).sum)
    val twoPop: Vector[Int] = dailyLockdownPicture("two").map(day => day.map(_.population).sum)
    Vector(zeroPop, onePop, twoPop)
}

val newCasesCurve: Vector[Vector[Double]] = {
    val zeroPop: Vector[Double] = dailyLockdownPicture("zero").map(day => { 
        val pop: Double = day.map(_.population).sum.toDouble
        val newCases: Double = day.map(_.newCases).sum.toDouble
        if (( (newCases * perCapitaNumber.toDouble) / pop).isNaN) 0
        else ((newCases * perCapitaNumber.toDouble) / pop)
       
    })
     val onePop: Vector[Double] = dailyLockdownPicture("one").map(day => { 
        val pop: Double = day.map(_.population).sum.toDouble
        val newCases: Double = day.map(_.newCases).sum.toDouble
        if (( (newCases * perCapitaNumber.toDouble) / pop).isNaN) 0
        else ((newCases * perCapitaNumber.toDouble) / pop)
    }) 
    val twoPop: Vector[Double] = dailyLockdownPicture("two").map(day => { 
        val pop: Double = day.map(_.population).sum.toDouble
        val newCases: Double = day.map(_.newCases).sum.toDouble
        if (( (newCases * perCapitaNumber.toDouble) / pop).isNaN) 0
        else ((newCases * perCapitaNumber.toDouble) / pop)
    })
    
    Vector(zeroPop, onePop, twoPop)
}

val newDeathsCurve: Vector[Vector[Double]] = {
    val zeroPop: Vector[Double] = dailyLockdownPicture("zero").map(day => { 
        val pop: Double = day.map(_.population).sum.toDouble
        val newDeaths: Double = day.map(_.newDeaths).sum.toDouble
        if (( (newDeaths * perCapitaNumber.toDouble) / pop).isNaN) 0
        else ((newDeaths * perCapitaNumber.toDouble) / pop)
       
    })
     val onePop: Vector[Double] = dailyLockdownPicture("one").map(day => { 
        val pop: Double = day.map(_.population).sum.toDouble
        val newDeaths: Double = day.map(_.newDeaths).sum.toDouble
        if (( (newDeaths * perCapitaNumber.toDouble) / pop).isNaN) 0
        else ((newDeaths * perCapitaNumber.toDouble) / pop)
    }) 
    val twoPop: Vector[Double] = dailyLockdownPicture("two").map(day => { 
        val pop: Double = day.map(_.population).sum.toDouble
        val newDeaths: Double = day.map(_.newDeaths).sum.toDouble
        if (( (newDeaths * perCapitaNumber.toDouble) / pop).isNaN) 0
        else ((newDeaths * perCapitaNumber.toDouble) / pop)
    })
    
    Vector(zeroPop, onePop, twoPop)
}

val popUnderLockdown: Unit = {
    val data: Vector[Vector[Double]] = populationCurve.map( pc => {
        pc.map(_.toDouble)
    })
    
    val sums: Vector[Double] = data(0).zipWithIndex.map( di => {
        val d = di._1
        val i = di._2
        (d + data(1)(i) + data(2)(i))
    })
    
    val allData: Vector[Vector[Double]] = data ++ Vector(sums)

    
    val labels = Vector("Open", "Partial", "Full", "Total")
    val title = s"Population under Lockdown"
    
    plotData(allData, labels, title )
}

val newCasesUnderLockdown: Unit = {
    // let's do a rolling average to smooth things out…
    val rollingAve: Int = 7 // change to 1 for raw data
    
    val data: Vector[Vector[Double]] = newCasesCurve.map( v => {
        v.sliding(rollingAve, 1).map( ra => ra.toVector.sum / rollingAve ).toVector
    })
    
    val sums: Vector[Double] = data(0).zipWithIndex.map( di => {
        val d = di._1
        val i = di._2
        (d + data(1)(i) + data(2)(i))
    })
    
    val allData: Vector[Vector[Double]] = data ++ Vector(sums)
    
    
    val labels = Vector("Open", "Partial", "Full", "Total")
    val title = s"New Cases Per ${perCapitaNumber} under Lockdown"
    
    plotData(allData, labels, title )
}

val newDeathsUnderLockdown: Unit = {
    // let's do a rolling average to smooth things out…
    val rollingAve: Int = 7 // change to 1 for raw data
    
    val data: Vector[Vector[Double]] = newDeathsCurve.map( v => {
        v.sliding(rollingAve, 1).map( ra => ra.toVector.sum / rollingAve ).toVector
    })
    
     val sums: Vector[Double] = data(0).zipWithIndex.map( di => {
        val d = di._1
        val i = di._2
        (d + data(1)(i) + data(2)(i))
    })
    
    val allData: Vector[Vector[Double]] = data ++ Vector(sums)
    
    
    val labels = Vector("Open", "Partial", "Full", "Total")
    val title = s"New Deaths Per ${perCapitaNumber} under Lockdown"
    
    plotData(allData, labels, title )
}

## Enough Averages: Let's See All States

Let's see how *all* Red States fared versus Blue States, and how all states, in our "lockdown-thirds" fared.

In [ ]:
// This uses a rolling average, parameterized below…

val allRedBlueDailyDeathsPerCap: Unit = {
    
   val rollingAve: Int = 7 // change to 1 for raw data
    
   
   //val redData: Vector[Double]  = dailyDeaths(redStates, true).sliding(rollingAve,1).toVector.map(_.sum / rollingAve)

   val redData: Vector[Vector[Double]] = redStates.states.map(_.days.map( d => {
       (d.newDeaths.toDouble * perCapitaNumber.toDouble) / d.population.toDouble 
   })).map( _.sliding(rollingAve, 1).toVector.map( n => n.sum / rollingAve))
                                                              
   val blueData: Vector[Vector[Double]] = blueStates.states.map(_.days.map( d => {
       (d.newDeaths.toDouble * perCapitaNumber.toDouble) / d.population.toDouble 
   })).map( _.sliding(rollingAve, 1).toVector.map( n => n.sum / rollingAve))
    
   val lables = lockdownThirds.toVector.map( _._1 )

    
    val redLabels: Vector[String] = redStates.states.map(_.days.head.state)
    val blueLabels: Vector[String] = blueStates.states.map(_.days.head.state)

    
    val title = s"Red v. Blue, Daily Deaths, per ${perCapitaNumber}, ${rollingAve}-day avg."
        
    val dataRed = getScatterVec( redData, redLabels, Some((240,0,0,0.5)))
    val dataBlue = getScatterVec( blueData, blueLabels, Some((0,0,240,0.2)))
    val dataNew = dataRed ++ dataBlue

    
    val layoutNew = Layout(title)

    plot(dataNew, layoutNew)


}



In [ ]:
// This uses a rolling average, parameterized below…

val allLockdDownThirdsDailyDeathsPerCap: Unit = {
    
   val rollingAve: Int = 7 // change to 1 for raw data
    
   
   //val redData: Vector[Double]  = dailyDeaths(redStates, true).sliding(rollingAve,1).toVector.map(_.sum / rollingAve)

   val lowData: Vector[Vector[Double]] = lockdownThirds("low").states.map(_.days.map( d => {
       (d.newDeaths.toDouble * perCapitaNumber.toDouble) / d.population.toDouble 
   })).map( _.sliding(rollingAve, 1).toVector.map( n => n.sum / rollingAve))
                                                              
    val middleData: Vector[Vector[Double]] = lockdownThirds("middle").states.map(_.days.map( d => {
       (d.newDeaths.toDouble * perCapitaNumber.toDouble) / d.population.toDouble 
   })).map( _.sliding(rollingAve, 1).toVector.map( n => n.sum / rollingAve))
    
    val highData: Vector[Vector[Double]] = lockdownThirds("high").states.map(_.days.map( d => {
       (d.newDeaths.toDouble * perCapitaNumber.toDouble) / d.population.toDouble 
   })).map( _.sliding(rollingAve, 1).toVector.map( n => n.sum / rollingAve))
    
   val lables = lockdownThirds.toVector.map( _._1 )

    
    val lowLabels: Vector[String] = lockdownThirds("low").states.map(_.days.head.state)
    val middleLabels: Vector[String] = lockdownThirds("middle").states.map(_.days.head.state)
    val highLabels: Vector[String] = lockdownThirds("high").states.map(_.days.head.state)



    
    val title = s"Daily Deaths, per ${perCapitaNumber}, ${rollingAve}-day avg."
        
    val dataLow = getScatterVec( lowData, lowLabels, Some((75,158,101,0.3)))
    val dataMiddle = getScatterVec( middleData, middleLabels, Some((240,154,56,1)))
    val dataHigh = getScatterVec( highData, middleLabels, Some((240,0,0,0.1)))


    val dataNew = (dataLow ++ dataMiddle ++ dataHigh).reverse

    
    val layoutNew = Layout(title)

    plot(dataNew, layoutNew)


}

